In [65]:
import os
from collections import defaultdict

import numpy as np

# Read ICML2014 dataset

In [2]:
data = np.load('data/cullpdb+profile_6133.npy.gz')

In [3]:
data.shape

(6133, 39900)

In [4]:
data = data.reshape((data.shape[0], 700, 57))

In [5]:
data.shape

(6133, 700, 57)

In [6]:
n_train = 5600
n_test = 5877-5605
n_val = 6133-5877

In [7]:
x_train = data[:n_train,:,:22]
x_test = data[n_train:n_train+n_test,:,:22]
x_val = data[n_train+n_test:,:,:22]

y_train = data[:n_train,:,22:31]
y_test = data[n_train:n_train+n_test,:,22:31]
y_val = data[n_train+n_test:,:,22:31]

In [8]:
labels_amino_acids = ['A', 'C', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'M', 'L', 'N', 'Q', 'P', 'S', 'R', 'T', 'W', 'V', 
                      'Y', 'X', None]
labels_sec_structure = ['L', 'B', 'E', 'G', 'I', 'H', 'S', 'T', None]

In [9]:
def one_hot_to_sequences(one_hot_vector, encoding):
    sequences = []
    for s in one_hot_vector:
        sequence = []
        for l in s:
            letter = encoding[np.where(l == 1)[0][0]]
            if letter is not None:
                sequence.append(letter)
        sequences.append(sequence)
    
    return sequences

In [10]:
x_train_sequences = one_hot_to_sequences(x_train, labels_amino_acids)
x_test_sequences = one_hot_to_sequences(x_test, labels_amino_acids)
x_val_sequences = one_hot_to_sequences(x_val, labels_amino_acids)

y_train_sequences = one_hot_to_sequences(y_train, labels_sec_structure)
y_test_sequences = one_hot_to_sequences(y_test, labels_sec_structure)
y_val_sequences = one_hot_to_sequences(y_val, labels_sec_structure)

# Predictions

## JPred4

In [11]:
import jpredapi

In [34]:
def reduction_8_to_3(seq):
    new_seq = []
    for state in seq:
        new_state = '-'
        if state in ('H', 'G'):
            new_state = 'H'
        elif state in ('E', 'B'):
            new_state = 'E'
            
        new_seq.append(new_state)
    
    return new_seq

In [44]:
def sequence_to_array(seq):
    arr = []
    for state in seq:
        state_idx = 0
        if state in ('H', 'G'):
            state_idx = 1
        elif state in ('E', 'B'):
            state_idx = 2
            
        arr.append(state_idx)
    
    return np.array(arr)

def sequence_accuracy(a, b):
    a, b = sequence_to_array(a), sequence_to_array(b)
    return np.mean(np.equal(a, b))

In [45]:
print(''.join(x_test_sequences[0]))
print(''.join(y_test_sequences[0]))

jpred_output = '-,-,-,H,H,H,H,H,H,H,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,E,E,E,E,-,-,-,-,' + \
    '-,-,-,-,-,-,E,E,E,-,-,-,-,-,'

print(''.join(reduction_8_to_3(y_test_sequences[0])))
print(''.join(jpred_output.split(',')[:-1]))

print('Accuracy: %s' % sequence_accuracy(y_test_sequences[0], jpred_output.split(',')[:-1]))

GTNAAMRKAFNYQDTAKNGKKCSGCAQFVPGASPTAAGGCKVIPGDNQIAPGGYCDAFIVKK                                                                                        
LLLHHHHHHTTLBSSLBTTBLGGGBTTEELLSSTTSLBEETTSTTLLLBLTTLBLTTLLBLL
---HHHHHH---E---E--E-HHHE--EE--------EEE--------E----E-----E--
---HHHHHHH------------------------------EEEE----------EEE-----
Accuracy: 0.6290322580645161


In [23]:
for seq in x_test_sequences[:10]:
    if len(seq) < 150:
        for _ in range(len(seq), 150):
            seq.append(' ')
    elif len(seq) > 150:
        continue

    print(''.join(seq))

GTNAAMRKAFNYQDTAKNGKKCSGCAQFVPGASPTAAGGCKVIPGDNQIAPGGYCDAFIVKK                                                                                        
AIEKRLASLLTGQGLAFRVQDASLPGRPDFVVDEYRCVIFTHGCFWHHHHCYLFKVPATRTEFWLEKIGKNVERDRRDISRLQELGWRVLIVWECALRGREKLTDEALTERLEEWICGEGASAQIDTQGIHLLA                
XSLVEATLEVIGGKWKXVILXHLTHGKKRTSELKRLXPNITQKXLTQQLRELEADGVINRIVYNQVPPKVEYELSEYGRSLEGILDXLXAWGANHINR                                                    
SNIGIRDLAVQFSCIEAVNMASKILKSYESSLPQTQQVDLDLSRPLFTSAALLSACKILKLKVDKNKMVATSGVKKAIFDRLCKQLEKIGQQV                                                         
ERRVRELTEATNDILWEFTADLSEVLVINSAYEDIWGRSVAKLRENPHDFLNGIHPEDRELMKDTMQSLMDGESADVECRVNATEEYQRWVWIQGEPITNDAGETVRVAGFARDIT                                  
ADERLQFTATTLSGAPFDGASLQGKPAVLWFWTPWCPFCNAEAPSLSQVAAANPAVTFVGIATRADVGAMQSFVSKYNLNFTNLNDADGVIWARYNVPWQPAFVFYRADGTSTFVNNPTAAMSQDELSGRVAAL                
GGVTDALSLXYSTSTGGPASIAANALTDFDLSGALTVNSVGTGLTKSAAGIQLAAGKSGLYQITXTVKNNTVTTGNYLLRVKYGSSDFVVACPA

In [20]:
len('AIEKRLASLLTGQGLAFRVQDASLPGRPDFVVDEYRCVIFTHGCFWHHHHCYLFKVPATRTEFWLEKIGKNVERDRRDISRLQELGWRVLIVWECALRGREKLTDEALTERLEEWICGEGASAQIDTQGIHLLA')

134

In [17]:
''.join(y_test_sequences[0])

'LLLHHHHHHTTLBSSLBTTBLGGGBTTEELLSSTTSLBEETTSTTLLLBLTTLBLTTLLBLL'

In [46]:
r = jpredapi.submit(mode='single', user_format='raw', seq=''.join(x_test_sequences[0]), skipPDB=True)

Your job will be submitted with the following parameters:
format: seq
skipPDB: on
seq: GTNAAMRKAFNYQDTAKNGKKCSGCAQFVPGASPTAAGGCKVIPGDNQIAPGGYCDAFIVKK                                                                                        
Created JPred job with jobid: jp_2bRS0l7
You can check the status of the job using the following URL: http://www.compbio.dundee.ac.uk/jpred4/cgi-bin/chklog?jp_2bRS0l7


In [15]:
jpredapi.status(job_id='jp_8uZHLfq')

Your job status will be checked with the following parameters:
Job id: jp_8uZHLfq
Get results: False
Job jp_8uZHLfq finished. Results available at the following URL:
http://www.compbio.dundee.ac.uk/jpred4/results/jp_8uZHLfq/jp_8uZHLfq.results.html




In [16]:
jpredapi.get_results(job_id='jp_8uZHLfq', results_dir_path='data/jpred_sspred/results', extract=True)

Your job status will be checked with the following parameters:
Job id: jp_8uZHLfq
Get results: True
Job jp_8uZHLfq finished. Results available at the following URL:
http://www.compbio.dundee.ac.uk/jpred4/results/jp_8uZHLfq/jp_8uZHLfq.results.html


Saving results to: /home/ubuntu/COMP3212/data/jpred_sspred/results/jp_8uZHLfq


In [14]:
len(x_test_sequences)

272

In [30]:
print(''.join(x_test_sequences[0]))
print(''.join(y_test_sequences[0]))
print('CCCHHHHHHEEEHHHHHTHTCCCCEEEEHHHCCCCCCCCCCCCCTCTCCCCTCEEHEEEHHC')

GTNAAMRKAFNYQDTAKNGKKCSGCAQFVPGASPTAAGGCKVIPGDNQIAPGGYCDAFIVKK
LLLHHHHHHTTLBSSLBTTBLGGGBTTEELLSSTTSLBEETTSTTLLLBLTTLBLTTLLBLL
CCCHHHHHHEEEHHHHHTHTCCCCEEEEHHHCCCCCCCCCCCCCTCTCCCCTCEEHEEEHHC


## JPred4 - working 🤗

In [62]:
x_test_sequences_filtered = []
y_test_sequences_filtered = []
for x_seq, y_seq in zip(x_test_sequences, y_test_sequences):
    if len(x_seq) <= 20:
        continue
        
    x_test_sequences_filtered.append(x_seq)
    y_test_sequences_filtered.append(y_seq)

In [63]:
for chunk_i, chunk in enumerate([(0, 100), (100, 200), (200, len(x_test_sequences_filtered))]):
    with open('data/cullpdb_x_test_sequences.%s.fasta' % chunk_i, 'w') as out_fasta:
        for i, seq in enumerate(x_test_sequences_filtered[chunk[0]:chunk[1]]):
            out_fasta.write('>sequence_%s\n' % (chunk[0] + i))
            out_fasta.write('%s\n' % ''.join(seq))

In [83]:
# jpredapi.submit(mode='batch', user_format='fasta', file='data/cullpdb_x_test_sequences.0.fasta', skipPDB=True,
#                 email='ms3u14@soton.ac.uk')
jpredapi.submit(mode='batch', user_format='fasta', file='data/cullpdb_x_test_sequences.1.fasta', skipPDB=True,
                email='ms3u14@soton.ac.uk')
# jpredapi.submit(mode='batch', user_format='fasta', file='data/cullpdb_x_test_sequences.2.fasta', skipPDB=True,
#                 email='ms3u14@soton.ac.uk')

Your job will be submitted with the following parameters:
format: batch
skipPDB: on
file: data/cullpdb_x_test_sequences.1.fasta
email: ms3u14@soton.ac.uk
b'<h1>\t\tYou have successfully submitted 100 sequences to Jpred as a batch job. You should receive your results at ms3u14@soton.ac.uk in due course.</h1>'


In [159]:
jpred_results = {}
for dir_ in os.listdir('data'):
    path_dir = 'data/%s/' % dir_
    if not dir_.startswith('jp_batch_') or not os.path.isdir(path_dir):
        continue
    
    result = None
    sequence = None
    for file in os.listdir(path_dir):
        if file.startswith('sequence_'):
            sequence = int(''.join([c for c in file[9:12] if c.isdigit()]))
            continue
        
        if file.endswith('.jnet'):
            with open(path_dir + file, 'r') as _in:
                for line in _in:
                    line_split = line.split(':')
                    if len(line_split) <= 0:
                        continue
                        
                    if line_split[0] == 'jnetpred':
                        result = line_split[1].split(',')[:-1]
                        continue
    
    if result is not None and sequence is not None:
        jpred_results[sequence] = result

In [160]:
for i in range(0, len(jpred_results)):
    print(len(jpred_results[i]), len(y_test_sequences_filtered[i]))

62 62
134 134
393 393
98 98
93 93
116 116
176 176
134 134
144 144
193 193
102 102
91 91
92 92
257 257
283 283
359 359
248 248
268 268
205 205
245 245
47 47
412 412
109 109
194 194
299 299
63 63
157 157
453 453
289 289
185 185
295 295
138 138
326 326
244 244
149 149
192 192
138 138
342 342
278 278
53 53
171 171
118 118
252 252
351 351
209 209
74 74
319 319
153 153
265 265
171 171
68 68
113 113
109 109
238 238
75 75
94 94
388 388
156 156
371 371
458 458
167 167
206 206
256 256
131 131
224 224
144 144
143 143
51 51
122 122
101 101
246 246
288 288
72 72
207 207
92 92
230 230
139 139
220 220
442 442
136 136
181 181
385 385
192 192
261 261
180 180
407 407
119 119
128 128
152 152
250 250
227 227
134 134
250 250
149 149
70 70
39 39
139 139
85 85
258 258
206 206
192 192
461 461
171 171
113 113
58 58
114 114
377 377
186 186
182 182
294 294
73 73
130 130
80 80
185 185
129 129
152 152
117 117
437 437
270 270
297 297
190 190
153 153
149 149
244 244
107 107
493 493
97 97
128 128
241 241
197 197
176 

In [161]:
len(jpred_results), len(y_test_sequences_filtered)

(271, 271)

In [162]:
means = []
for sequence_i, result in jpred_results.items():
    sequence_to_array(y_test_sequences_filtered[sequence_i])
    means.append(np.mean(np.equal(sequence_to_array(y_test_sequences_filtered[sequence_i]), 
                                  sequence_to_array(result))))

np.mean(means)

0.7958423358878534

In [129]:
sequence_to_array(y_test_sequences_filtered[sequence_i])

array([0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2,

In [130]:
sequence_to_array(result)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0])

## SCRATCH

### Generate FASTA file 

In [ ]:
with open('data/cullpdb_x_test_sequences.fasta', 'w') as out_fasta:
    for i, seq in enumerate(x_test_sequences):
        out_fasta.write('>sequence_%s\n' % i)
        out_fasta.write('%s\n' % ''.join(seq))
        
        print('>sequence_%s' % i)
        print('%s' % ''.join(seq))

In [27]:
x_test_sequences_lenghts = defaultdict(int)
for seq in x_test_sequences:
    x_test_sequences_lenghts[len(seq)] += 1

In [29]:
sum(x_test_sequences_lenghts) / len(x_test_sequences_lenghts)

224.09947643979058

In [32]:
y_test[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])